In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden MediaPipe with text classification

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_mediapipe_text_classification.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_mediapipe_image_classification.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

**NOTE**: The checkpoint and the dataset linked in this Colab are not owned or distributed by Google, and are made available by third parties. Please review the terms and conditions made available by the third parties before using the checkpoint and data.

## Overview

This notebook demonstrates how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train an on-device text classification model in Vertex AI Model Garden.

### Objective

* Train new models
  * Convert input data to training formats
  * Create [custom jobs](https://cloud.google.com/vertex-ai/docs/training/create-custom-job) to train new models
  * Export models

* Cleanup resources

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

### Colab only
Run the following commands to install dependencies and to authenticate with Google Cloud if running on Colab.

In [ ]:
! pip3 install --upgrade pip

import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
import json
import os
from datetime import datetime

from google.cloud import aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
now = datetime.now().strftime("%Y%m%d-%H%M%S")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temp/%s" % now)

EVALUATION_RESULT_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "evaluation")
EVALUATION_RESULT_OUTPUT_FILE = os.path.join(
    EVALUATION_RESULT_OUTPUT_DIRECTORY, "evaluation.json"
)

EXPORTED_MODEL_OUTPUT_DIRECTORY = os.path.join(STAGING_BUCKET, "model")
EXPORTED_MODEL_OUTPUT_FILE = os.path.join(
    EXPORTED_MODEL_OUTPUT_DIRECTORY, "model.tflite"
)

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### Define training machine specs

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "mediapipe_text_classifier_%s" % now
TRAINING_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/mediapipe-train"
TRAINING_MACHINE_TYPE = "n1-highmem-16"
TRAINING_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAINING_ACCELERATOR_COUNT = 2

## Train your customized models

### Get the Dataset

The following code block uses the [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) dataset which contains 67,349 movie reviews for training and 872 movie reviews for testing. The dataset has two classes: positive and negative movie reviews. Positive reviews are labeled with 1 and negative reviews with 0.

The SST-2 dataset is stored as a TSV file. The only difference between the TSV and CSV formats is that TSV uses a tab `\t` character as its delimiter and CSV uses a comma `,`.


In [ ]:
training_data_path = (
    "gs://mediapipe-tasks/text_classifier/SST-2/train.tsv"  # @param {type:"string"}
)
validation_data_path = (
    "gs://mediapipe-tasks/text_classifier/SST-2/dev.tsv"  # @param {type:"string"}
)

# The delimiter used in the dataset.
delimiter = "\t"  # @param {type:"string"}

# Character used to quote fields that contain special characters
# like the `delimiter`.
quotechar = "\t"  # @param {type:"string"}

# Sequence of keys for the CSV columns (represented as a comma
# separated list). If empty, the first row of the CSV file is used
# as the keys
fieldnames = ""  # @param {type:"string"}

# Column name for the input text.
text_column = "sentence"  # @param {type:"string"}

# Column name for the labels.
label_column = "label"  # @param {type:"string"}

### Set fine-tuning options

You can pick between different model architectures to further customize your training:

*   Average Word Embedding Model
*   BERT-classifier

To set the model architecture and other training parameters, adjust the following values:

In [ ]:
model_architecture = (
    "average_word_embedding"  # @param ["average_word_embedding", "mobilebert"]
)

# The learning rate to use for gradient descent-based
# optimizers. Defaults to 3e-5 for the BERT-based classifier
# and 0 for the average word-embedding classifier because
# it does not need such an optimizer.
learning_rate: float = 0.0  # @param {type:"number"}

# Batch size for training. Defaults to 32 for the average
# word-embedding classifier and 48 for the BERT-based
# classifier.
batch_size: int = 48  # @param {type:"number"}

# Number of training iterations over the dataset. Defaults
# to 10 for the average word-embedding classifier and 3
# for the BERT-based classifier.
epochs: int = 10  # @param {type:"slider", min:0, max:100, step:1}

# An integer that indicates the number of training steps per
# epoch. If set to 0, the training pipeline calculates the
# default steps per epoch as the training dataset size
# divided by batch size.
steps_per_epoch: int = 0  # @param {type:"number"}

# Controls whether the dataset is shuffled before training.
shuffle: bool = False  # @param {type:"boolean"}

# Length of the sequence to feed into the model.
seq_len: int = 256  # @param {type:"number"}

# Whether to convert all uppercase characters to lowercase
# during preprocessing.
do_lower_case: bool = True  # @param {type:"boolean"}

# The rate for dropout.
dropout_rate: float = 0.2  # @param {type:"number"}

# Dimension of the word embedding. Only used for the Average Word
# Embedding Model.
wordvec_dim: int = 16  # @param {type:"number"}

# Number of words to generate the vocabulary from data.
# Only used for the Average Word Embedding Model.
vocab_size: int = 10000  # @param {type:"number"}

### Run fine-tuning
With your training dataset and fine-tuning options prepared, you are ready to start the fine-tuning process. This process is resource intensive and can take a few minutes to a few hours depending on the model archtiecture and your available compute resources. On Vertex AI with GPU processing, the example fine-tuning below takes between 2-3 minutes to train an Average Word Embedding Model on the SST-2 dataset.

To begin the fine-tuning process, use the following code:


In [ ]:
model_export_path = EXPORTED_MODEL_OUTPUT_DIRECTORY
evaluation_result_path = EVALUATION_RESULT_OUTPUT_DIRECTORY

preprocessing_params = {
    "text_column": text_column,
    "label_column": label_column,
    "delimiter": delimiter,
    "quotechar": quotechar,
}
if fieldnames:
    preprocessing_params["fieldnames"] = [
        fieldname.strip() for fieldname in fieldnames.split(",")
    ]

hparams = {
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "epochs": epochs,
    "shuffle": shuffle,
}
if steps_per_epoch:
    hparams["steps_per_epoch"] = steps_per_epoch

model_options = {
    "dropout_rate": dropout_rate,
    "wordvec_dim": wordvec_dim,
    "do_lower_case": do_lower_case,
    "vocab_size": vocab_size,
    "dropout_rate": dropout_rate,
}

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAINING_MACHINE_TYPE,
            "accelerator_type": TRAINING_ACCELERATOR_TYPE,
            "accelerator_count": TRAINING_ACCELERATOR_COUNT,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAINING_CONTAINER,
            "command": [],
            "args": [
                "--task_name=text_classifier",
                "--training_data_path=%s" % training_data_path,
                "--validation_data_path=%s" % validation_data_path,
                "--evaluation_result_path=%s" % evaluation_result_path,
                "--model_export_path=%s" % model_export_path,
                "--model_architecture=%s" % model_architecture,
                "--preprocessing_params=%s" % json.dumps(preprocessing_params),
                "--hparams=%s" % json.dumps(hparams),
                "--model_options=%s" % json.dumps(model_options),
            ],
        },
    }
]

training_job = aiplatform.CustomJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

training_job.run()

## Export model

After finetuning, you can save the Tensorflow Lite model, try it out in the [Text Classification](https://mediapipe-studio.webapps.google.com/demo/text_classifier) demo in MediaPipe Studio or integrate it with your on-device application by following the [Text classification task guide](https://developers.google.com/mediapipe/solutions/text/text_classifier). The exported model contains the generates required model metadata, as well as a classification label file.

In [ ]:
import sys


def copy_model(model_source, model_dest):
    ! gsutil cp {model_source} {model_dest}

copy_model(EXPORTED_MODEL_OUTPUT_FILE, "text_classification_model.tflite")

if "google.colab" in sys.modules:
    from google.colab import files

    files.download("text_classification_model.tflite")

## Clean up

In [ ]:
# Delete training data and jobs.
if training_job.list(filter=f'display_name="{TRAINING_JOB_DISPLAY_NAME}"'):
    training_job.delete()

!gsutil rm -r {STAGING_BUCKET}